# Normalização Prévia - Camada Prata

Este notebook implementa a normalização léxica dos endereços, expandindo abreviações, tratando numerais e corrigindo problemas de fonética e ortografia.

In [ ]:
# Importar configurações
%run ./00_configuracao_inicial.ipynb

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import re
import unicodedata

# Dicionário de abreviações comuns em endereços brasileiros
ABREVIACOES = {
    # Tipos de logradouro
    'r\.': 'RUA',
    'r ': 'RUA',
    'rua': 'RUA',
    'av\.': 'AVENIDA',
    'av ': 'AVENIDA',
    'avenida': 'AVENIDA',
    'est\.': 'ESTRADA',
    'est ': 'ESTRADA',
    'estrada': 'ESTRADA',
    'rod\.': 'RODOVIA',
    'rodovia': 'RODOVIA',
    'trav\.': 'TRAVESSA',
    'tv\.': 'TRAVESSA',
    'travessa': 'TRAVESSA',
    'al\.': 'ALAMEDA',
    'alameda': 'ALAMEDA',
    'pc\.': 'PRAÇA',
    'pra\s*ca': 'PRAÇA',
    'praça': 'PRAÇA',
    
    # Títulos e qualificadores
    'dr\.': 'DOUTOR',
    'dr ': 'DOUTOR',
    'doutor': 'DOUTOR',
    'prof\.': 'PROFESSOR',
    'prof ': 'PROFESSOR',
    'professor': 'PROFESSOR',
    'eng\.': 'ENGENHEIRO',
    'eng ': 'ENGENHEIRO',
    'engenheiro': 'ENGENHEIRO',
    's\.': 'SANTO',
    'santa': 'SANTA',
    'são': 'SÃO',
    'sao': 'SÃO',
    
    # Palavras comuns
    'n\s*': 'NUMERO',
    'n[úu]mero': 'NUMERO',
    'n[º°]': 'NUMERO',
    'b\.': 'BAIRRO',
    'bairro': 'BAIRRO',
}

# Dicionário de numerais por extenso
NUMEROS_EXTENSO = {
    '1': 'UM', '2': 'DOIS', '3': 'TRES', '4': 'QUATRO', '5': 'CINCO',
    '6': 'SEIS', '7': 'SETE', '8': 'OITO', '9': 'NOVE', '10': 'DEZ',
    '11': 'ONZE', '12': 'DOZE', '13': 'TREZE', '14': 'QUATORZE', '15': 'QUINZE',
    '16': 'DEZESSEIS', '17': 'DEZESSETE', '18': 'DEZOITO', '19': 'DEZENOVE', '20': 'VINTE',
    'I': 'PRIMEIRO', 'II': 'SEGUNDO', 'III': 'TERCEIRO', 'IV': 'QUARTO', 'V': 'QUINTO',
    'VI': 'SEXTO', 'VII': 'SETIMO', 'VIII': 'OITAVO', 'IX': 'NONO', 'X': 'DECIMO',
    'XV': 'QUINZE', 'XX': 'VINTE', 'XXX': 'TRINTA',
}

# Correções fonéticas comuns
CORRECOES_FONETICAS = {
    'souza': 'SOUSA',
    'sousa': 'SOUSA',
    'silva': 'SILVA',
    'santos': 'SANTOS',
    'santana': 'SANTANA',
}

print("Dicionários de normalização carregados!")

In [ ]:
# UDF para remover acentos
def remover_acentos(texto):
    """Remove acentos de uma string"""
    if not texto:
        return None
    
    # Normaliza para NFD e remove diacríticos
    texto_nfd = unicodedata.normalize('NFD', texto)
    texto_sem_acentos = ''.join(char for char in texto_nfd if unicodedata.category(char) != 'Mn')
    
    return texto_sem_acentos

udf_remover_acentos = udf(remover_acentos, StringType())

In [ ]:
# UDF para normalizar texto (maiúsculas, remover espaços extras)
def normalizar_texto(texto):
    """Normaliza texto: maiúsculas, remove espaços extras"""
    if not texto:
        return None
    
    texto_normalizado = texto.upper().strip()
    texto_normalizado = re.sub(r'\s+', ' ', texto_normalizado)  # Múltiplos espaços -> um espaço
    texto_normalizado = re.sub(r'[^A-Z0-9\s]', '', texto_normalizado)  # Remove caracteres especiais
    
    return texto_normalizado

udf_normalizar_texto = udf(normalizar_texto, StringType())

In [ ]:
# UDF para expandir abreviações
def expandir_abreviacoes(texto):
    """Expande abreviações comuns em endereços"""
    if not texto:
        return None
    
    texto_expandido = texto
    
    # Ordenar por tamanho (maior primeiro) para evitar substituições parciais
    abreviacoes_ordenadas = sorted(ABREVIACOES.items(), key=lambda x: len(x[0]), reverse=True)
    
    for abreviacao, expansao in abreviacoes_ordenadas:
        # Usar word boundary para evitar substituições parciais
        padrao = r'\b' + abreviacao + r'\b'
        texto_expandido = re.sub(padrao, expansao, texto_expandido, flags=re.IGNORECASE)
    
    return texto_expandido

udf_expandir_abreviacoes = udf(expandir_abreviacoes, StringType())

In [ ]:
# UDF para converter numerais
def converter_numerais(texto):
    """Converte numerais romanos e arábicos para extenso quando apropriado"""
    if not texto:
        return None
    
    texto_convertido = texto
    
    # Converter numerais romanos comuns em nomes de logradouros
    # Ex: XV de Novembro -> QUINZE DE NOVEMBRO
    for numeral, extenso in NUMEROS_EXTENSO.items():
        if len(numeral) > 1:  # Apenas numerais compostos
            padrao = r'\b' + numeral + r'\b'
            texto_convertido = re.sub(padrao, extenso, texto_convertido, flags=re.IGNORECASE)
    
    return texto_convertido

udf_converter_numerais = udf(converter_numerais, StringType())

In [ ]:
# UDF para corrigir problemas fonéticos e ortográficos
def corrigir_fonetica(texto):
    """Corrige problemas fonéticos e ortográficos comuns"""
    if not texto:
        return None
    
    texto_corrigido = texto
    
    for erro, correcao in CORRECOES_FONETICAS.items():
        padrao = r'\b' + erro + r'\b'
        texto_corrigido = re.sub(padrao, correcao, texto_corrigido, flags=re.IGNORECASE)
    
    # Correções específicas comuns
    texto_corrigido = re.sub(r'\bSOUZA\b', 'SOUSA', texto_corrigido)
    texto_corrigido = re.sub(r'\bSAO\b', 'SÃO', texto_corrigido)
    
    return texto_corrigido

udf_corrigir_fonetica = udf(corrigir_fonetica, StringType())

In [ ]:
# Função principal de normalização
def normalizar_endereco(df_enderecos_estruturados):
    """
    Normaliza endereços estruturados aplicando todas as transformações
    
    Args:
        df_enderecos_estruturados: DataFrame com endereços estruturados
    
    Returns:
        DataFrame com endereços normalizados
    """
    df_normalizado = df_enderecos_estruturados \
        .withColumn("tipo_logradouro_norm", 
            udf_expandir_abreviacoes(udf_normalizar_texto(col("tipo_logradouro")))) \
        .withColumn("nome_logradouro_norm", 
            udf_corrigir_fonetica(
                udf_converter_numerais(
                    udf_expandir_abreviacoes(
                        udf_normalizar_texto(col("nome_logradouro"))
                    )
                )
            )
        ) \
        .withColumn("bairro_norm", 
            udf_corrigir_fonetica(
                udf_expandir_abreviacoes(
                    udf_normalizar_texto(col("bairro"))
                )
            )
        ) \
        .withColumn("complemento_norm", 
            udf_normalizar_texto(col("complemento"))) \
        .withColumn("endereco_normalizado", 
            concat_ws(" | ", 
                col("tipo_logradouro_norm"),
                col("nome_logradouro_norm"),
                col("numero"),
                col("bairro_norm"),
                col("complemento_norm")
            )
        ) \
        .withColumn("normalizado_em", current_timestamp()) \
        .select(
            col("id"),
            col("endereco_livre"),
            col("endereco_estruturado"),
            col("uf"),
            col("cidade"),
            col("tipo_logradouro"),
            col("nome_logradouro"),
            col("tipo_logradouro_norm"),
            col("nome_logradouro_norm"),
            col("numero"),
            col("bairro"),
            col("bairro_norm"),
            col("complemento"),
            col("complemento_norm"),
            col("endereco_normalizado"),
            col("normalizado_em")
        )
    
    return df_normalizado

In [ ]:
# Exemplo de uso
# df_enderecos_estruturados = read_delta_table(PATH_ENDERECOS_ESTRUTURADOS)
# df_normalizado = normalizar_endereco(df_enderecos_estruturados)

# Dados de exemplo
from pyspark.sql.types import StructType, StructField, StringType, TimestampType

schema_exemplo = StructType([
    StructField("id", StringType(), True),
    StructField("endereco_livre", StringType(), True),
    StructField("endereco_estruturado", StringType(), True),
    StructField("uf", StringType(), True),
    StructField("cidade", StringType(), True),
    StructField("tipo_logradouro", StringType(), True),
    StructField("nome_logradouro", StringType(), True),
    StructField("numero", StringType(), True),
    StructField("bairro", StringType(), True),
    StructField("complemento", StringType(), True),
])

dados_exemplo = [
    ("1", "r. do ouvidor, 120", "RUA | do ouvidor | 120 | centro | None", "RJ", "Rio de Janeiro", "RUA", "do ouvidor", "120", "centro", None),
    ("2", "Av. Paulista", "AVENIDA | Paulista | None | Bela Vista | None", "SP", "São Paulo", "AVENIDA", "Paulista", None, "Bela Vista", None),
    ("3", "Rua XV de Nov", "RUA | XV de Nov | None | centro | None", "SP", "São Paulo", "RUA", "XV de Nov", None, "centro", None),
]

df_exemplo = spark.createDataFrame(dados_exemplo, schema_exemplo)
df_exemplo.show(truncate=False)

In [ ]:
# Aplicar normalização
df_normalizado = normalizar_endereco(df_exemplo)
df_normalizado.show(truncate=False)

In [ ]:
# Salvar endereços normalizados
# save_delta_table(df_normalizado, PATH_ENDERECOS_NORMALIZADOS, mode="append", partition_by=["uf", "cidade"])

print("Processo de normalização concluído!")